# Design Thinking for Document Data Exploration from the Prototype code
**Group 6-Members**
* 110C51505 張仁程
* 110C51537 王星文
* 110C51522 謝安
* 110C51526	蔡侑芯
* 110C51533	陳毓庭
* 110C51520	楊文均

In [ ]:
#Using this if u'r on windows or linux env.
#pip install gdown.
import gdown
import os

path = "patent_documents"
target_folder = os.path.join("/ellom/NTUTDS", path)

if os.path.exists(target_folder):
    print(target_folder + "exist")
else:
    os.mkdir(target_folder)

url = 'https://drive.google.com/uc?export=download&id=10G2D9fm3UL92W2UuKdrw4bzJkDu0OflJ'
output = '/ellom/NTUTDS/patent_documents/patentdoc.tar.gz'
gdown.download(url, output, quiet=False)

In [10]:
import tarfile

def tar_extract(file_path):
    tar = tarfile.open(file_path, 'r:gz')
    tar.extractall()
    tar.close()

if __name__ == '__main__':
    file_path = output
    tar_extract(file_path)

In [ ]:
import pprint
from bs4 import BeautifulSoup
path = "patent_documents"
patent_docs_txt = {}
for filename in os.listdir(path):
  if filename.endswith(".txt"):
    with open(os.path.join(path, filename), 'r') as f:
      soup = BeautifulSoup(f.read(), 'xml')
      tag=soup.find_all()
      patent_docs_txt[filename] = tag[0].text.lower().replace('\n',' ')

pprint.pprint(patent_docs_txt, depth=1, width=10000)

In [ ]:
import nltk
import re

print(nltk.__version__)
nltk.download('punkt')
nltk.download('stopwords')
nltk_stopwords = nltk.corpus.stopwords.words('english')

def Patent_KW50(patent_text):
  tokens = nltk.word_tokenize(patent_text)
  fdist = nltk.FreqDist(tokens)
  common50 = fdist.most_common(50)
  common50_new1 = [(k,v) for k,v in common50 if not re.fullmatch(r"^.$", k)]
  common50_new2 = [(k,v) for k,v in common50_new1 if re.fullmatch(r"^[a-z]+$", k, flags=re.IGNORECASE)]
  stopword = list(nltk_stopwords)[:100]
  common50_new3 = dict([(k,v) for k,v in common50_new2 if not k in stopword])
  return common50_new3

patent_docs_KW = {}
for doc_name, txt in patent_docs_txt.items():
  patent_docs_KW[doc_name] = Patent_KW50(txt)

pprint.pprint(patent_docs_KW, width=1000)

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from itertools import islice

plt.clf()

for doc_name, KW_stat in patent_docs_KW.items():
  common20 = dict(islice(KW_stat.items(), 20))
  print(common20)
  x = np.arange(20)
  y = common20.values()
  plt.title( doc_name, fontsize=8, color='g')
  plt.xticks(x, common20.keys(),  rotation='vertical')
  plt.bar(x, y, color='y')
  plt.show()